---
# 올리브영 로션 제품군 크롤링
* "https://www.oliveyoung.co.kr/store/display/getMCategoryList.do?dispCatNo=100000100010016&fltDispCatNo=&prdSort=01&pageIdx=3"
---

In [1]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import requests
import time
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

# 전체 페이지

In [2]:
service = Service()
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=service,options=options)

time.sleep(0.2)

prd_items = []
prd_brands = []
prd_names = []
prd_prices = []
prd_infos = []
prd_thumbs = [] 
try: 
    for page in range(1, 3):
        url = f"https://www.oliveyoung.co.kr/store/display/getMCategoryList.do?dispCatNo=100000100010016&fltDispCatNo=&prdSort=01&pageIdx={page}"
        driver.get(url)

        time.sleep(1.0)
        prd_list = driver.find_element(By.ID,"Contents")
        # prd = prd_list.find_element(By.CLASS_NAME,"cate_prd_list")      # 한 페이지에 이 클래스 이름을 가진 ul이 여러 개고 그 ul마다 각각 상품 네개씩 있음
        prd = prd_list.find_elements(By.CLASS_NAME,"prd_info")              # 그냥 바로 상품 각각 찾기

        for i in range(len(prd)):
            prds = driver.find_elements(By.CLASS_NAME, "prd_info")  # Refresh the list of prd elements
            each_prd = prds[i]
            each_prd.click()

            info_btn = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "buyInfo")))
            info_btn.click()

            prd_items = driver.find_element(By.CLASS_NAME, "right_area")
            prd_brands.append(prd_items.find_element(By.ID, "moveBrandShop").text)    
            prd_name = prd_items.find_element(By.CLASS_NAME, "prd_name").text
            prd_names.append(prd_name)
            prd_prices.append(prd_items.find_element(By.CLASS_NAME, "price").text.split('\n')[0])
            
            
            prd_images_area = driver.find_element(By.ID, "prd_thumb_list")
            thumbs = prd_images_area.find_elements(By.TAG_NAME, "li")
            imageNames = []
            for idx, li in enumerate(thumbs):
                
                imgs = li.find_element(By.TAG_NAME, "img")
                imgsrc = imgs.get_attribute("src")
                imgres = requests.get(imgsrc)

                imageName = f"./prdimgs/page_{page}_prd_{i}_thumb{idx + 1}.jpg"
                with open(imageName, "wb") as f:
                    f.write(imgres.content)
                imageNames.append(imageName)
                
            prd_thumbs.append(imageNames)
            prd_info = WebDriverWait(driver,10).until(EC.visibility_of_element_located((By.ID, "artcInfo")))
            prd_infos.append(prd_info.text.split("\n"))
            driver.back()
            time.sleep(0.2)
finally:
    driver.quit()

In [3]:
# page=2까지
import pandas as pd
lotion = pd.DataFrame({"제품명":prd_names, "브랜드":prd_brands, "가격":prd_prices,'상품정보':prd_infos,'썸네일':prd_thumbs})
lotion

,제품명,브랜드,가격,상품정보,썸네일
0,아벤느 트릭세라 로션 (본품200ml+100ml 추가 증정),아벤느,"33,000","[내용물의 용량 또는 중량 정품: 200ml / 증정품: 100ml, 제품 주요 사...","[./prdimgs/page_1_prd_0_thumb1.jpg, ./prdimgs/..."
1,[NEW/리뉴얼] 에스트라 아토베리어365 로션 150ml,에스트라,"33,000","[내용물의 용량 또는 중량 150ml, 제품 주요 사양 모든 피부용, 사용기한(또는...",[./prdimgs/page_1_prd_1_thumb1.jpg]
2,라운드랩 1025 독도 로션 400ml 대용량,라운드랩,"39,000","[내용물의 용량 또는 중량 400ml, 제품 주요 사양 모든 피부용, 사용기한(또는...","[./prdimgs/page_1_prd_2_thumb1.jpg, ./prdimgs/..."
3,[대용량] 닥터지 더모이스처 배리어 D 로션 300ml 기획 (+멀티밤 1g),닥터지,"37,000","[내용물의 용량 또는 중량 로션 300ml + 밤 1g, 제품 주요 사양 모든 피부...","[./prdimgs/page_1_prd_3_thumb1.jpg, ./prdimgs/..."
4,[한정기획] 피지오겔 DMT 페이셜 로션 200ml+50ml 증정 기획,피지오겔,"38,000","[내용물의 용량 또는 중량 200ml+50ml, 제품 주요 사양 모든피부용.., 사...","[./prdimgs/page_1_prd_4_thumb1.jpg, ./prdimgs/..."
5,우르오스 올인원 200ml 2종 택 1,우르오스,"29,700","[내용물의 용량 또는 중량 ■ 200ml, 제품 주요 사양 ■ 중건성 피부를 위한 ...",[./prdimgs/page_1_prd_5_thumb1.jpg]
6,우르오스 스킨밀크 200ml,우르오스,"29,700","[내용물의 용량 또는 중량 ■ 200ml, 제품 주요 사양 ■ 중건성 피부를 위한 ...",[./prdimgs/page_1_prd_6_thumb1.jpg]
7,피지오겔 레드수딩 AI 진정보습 로션 200ml,피지오겔,"37,500","[내용물의 용량 또는 중량 200ml, 제품 주요 사양 모든 피부용, 사용기한(또는...",[./prdimgs/page_1_prd_7_thumb1.jpg]
8,싸이닉 더 심플 데일리 로션 260ml,싸이닉,"18,000","[내용물의 용량 또는 중량 260ml, 제품 주요 사양 모든피부용, 사용기한(또는 ...","[./prdimgs/page_1_prd_8_thumb1.jpg, ./prdimgs/..."
9,우르오스 스킨로션 200ml,우르오스,"29,700","[내용물의 용량 또는 중량 ■ 200ml, 제품 주요 사양 ■ 지복합성 피부를 위한...",[./prdimgs/page_1_prd_9_thumb1.jpg]


In [4]:
lotion.to_csv('./output/lotion_items.csv',index=False)

In [5]:
lotion.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48 entries, 0 to 47
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   제품명     48 non-null     object
 1   브랜드     48 non-null     object
 2   가격      48 non-null     object
 3   상품정보    48 non-null     object
 4   썸네일     48 non-null     object
dtypes: object(5)
memory usage: 2.0+ KB
